In [12]:
import pandas as pd
import math 
data = pd.read_csv("main_task.xls") 

In [13]:
data.columns=['Restaurant_id','City','Cuisine_Style','Ranking','Rating','Price_Range','Number_of_Reviews','Reviews','URL_TA','ID_TA']

In [14]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    city = df_output.City.unique()
    city_reviews = {}
    for i in city:
        city_reviews[i] = df_output[df_output.City == i]['Number_of_Reviews'].mean()
    df_output['Number_of_Reviews'] = df_output[['City','Number_of_Reviews']].apply(lambda x: city_reviews[x[0]] if pd.isna(x[1]) else x[1],axis=1)
    # тут ваш код по обработке NAN
    df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number_of_Reviews']).astype('uint8')
    df_output['Cuisine_Style_NAN'] = pd.isna(df_output['Cuisine_Style']).astype('uint8')
    df_output['Price_Range_NAN'] = pd.isna(df_output['Price_Range']).astype('uint8')
    
    
    # ################### 3. Encoding ############################################################## 
    # Параметр город
    city = {}
    for i in df_output.City:
        if i in city:
            city[i] += 1
        else:
            city[i] = {}
            city[i] = 1
    # Пyсть критерием будет таков: Если Лондон и Париж  - 3 (> 4000),  если 3000 -1000 (эти города) то - 2, остальные 1 
    def fun_3(line):
        if city[line] > 3500 :
            return 3
        elif city [line] > 1000:
            return 2
        else:
            return 1
    df_output['popular_City'] = df_output.City.apply(fun_3)
    # Параметр город
    df_output = pd.concat([df_output,pd.get_dummies(data.City)],axis=1)
    
    
    

    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    # Параметр количество кухонь
    # Введем параметр количество кухонь в данном ресторане
    def fun(line): # преобразование в списки столбца Cuisine_Style
        import re
        pattern = re.compile('\w+\s*&?\s*\w+\s*&?\s*\w+\s*&?\s*')
        #print(line)
        if line != line or line =='':
            return ''
        else:
            return pattern.findall(line)
    df_output.Cuisine_Style = df_output.Cuisine_Style.apply(fun)

    df_output['number_Cuisine'] =  df_output.Cuisine_Style.apply(lambda x: 1 if len(x) == 0 else len(x))
      
    # Создаим one hot encoding для кухонь
    spisok = []
    for j in df_output.Cuisine_Style:
        if j==j:
            for i, value in enumerate (j):
                if value not in spisok:
                    spisok.append(value)
    for i in spisok:
        str_ = 'Style_' + i
        df_output[str_] = 0
        
    for i, value in enumerate (df_output.Cuisine_Style):
        for j in value:
            str_ = str('Style_') + j
            df_output[str_][i] = 1
    

    # Добавим параметр популярности ресторана по типу есть ли  внем популярная кухня 'Vegetarian Friendly'
    # Будем исходить из предположения, что елси данных о кухонь нет, то ресторан не очень популярный и врядтли
    #  у них популярные веганские блюда. Пусть если есть данная кухня то 3,если нет кухни то 2, иначе 1
    def fun_2(line):
        if 'Vegetarian Friendly' in line:
            return 3
        elif len(line) != 0 :
            return 2
        else:
            return 1
    df_output['popular_Cuisine'] = df_output.Cuisine_Style.apply(fun_2)
    
    # Рассмотрим параметр дорогой/средний/дешевый ресторан по параметру Price_Range, если данных нет то заполним
    #  средними значениями: 1 - дешевый, 2 - средний, 3 дорогой
    def fun_1(line):
        if line == '$$ - $$$':
            return 2
        elif line == '$':
            return 1
        elif line == '$$$$':
            return 3
        else:
            return 2 # сложный вопрос так как в ровно половине данных отсутствует значение
    # поэтому в регрессии этот вклад будет превалировать, возможно стоит раскидать значения рандомно вместо удаления 

    df_output['Price'] = df_output.Price_Range.apply(fun_1)
    
    # Введем параметр показывающий отклонение от максимума разницу между отзывами в ресторане
    # Если ресторан хороший, то он должен следить за своей репутацией, соответственно если данных нет
    # ресторан плохой и не следит за своим менеджментом
    # Критерий max = 3207, смотреть будем по модулю
    # если значение большое это хорошо, если значение маленькое значит плохо
    # ТО есть чем больше значение тем лучше работает ресторан, если данных нет то значение ставим 0
    df_output['time_1']=''
    df_output['time_2']=''
    #line = "[['Good food at your doorstep', 'A good hotel restaurant'], ['12/31/2017', '11/20/2017']]"
    for i, line in enumerate (df_output.Reviews):
        import re
        from datetime import datetime, timedelta
        pattern = re.compile('\d+/\d+/\d+') 
        date_string = pattern.findall(str(line))
        if len(date_string) == 0:  
            df_output['time_1'][i] = ''
            df_output['time_2'][i] = ''
        elif len(date_string) == 1:
            df_output['time_1'][i] = datetime.strptime(date_string[0], '%m/%d/%Y')
            df_output['time_2'][i] = ''
        elif len(date_string) == 2:
            df_output['time_1'][i] = datetime.strptime(date_string[0], '%m/%d/%Y')
            df_output['time_2'][i] = datetime.strptime(date_string[1], '%m/%d/%Y')
   # import math 
   # import numpy as np
    df_output['diff'] = 0
    max_ = 3207

    for i in range(len(df_output)):
      #  print(i)
        if df_output.time_1[i] != '' and df_output.time_2[i] != '' :
            diff_ = df_output.time_1[i] - df_output.time_2[i]
            df_output['diff'][i] = max_ - math.fabs(diff_.days)
        else:
            df_output['diff'][i] = 0
   
    # данные говорят о том, что некоторые ID имеют более 1 значения, то есть можно сделать вывод, что это сетевые рестораны
    chain_restor = df_output.Restaurant_id.value_counts()[df_output.Restaurant_id.value_counts() > 1].index.tolist()
    df_output['chain_restor'] = df_output[df_output.Restaurant_id.isin(chain_restor)].Restaurant_id.apply(lambda x: 1)
    df_output.chain_restor=df_output['chain_restor'].fillna(0)

    # нормировка значений Ranking
    city = df_output.City.unique()
    City_ranking = {}
    for i in city:
        City_ranking[i] = df_output[data.City == i].Ranking.max()
    df_output['Ranking_NORM'] = df_output[['City','Ranking']].apply(lambda x: x[1]/City_ranking[x[0]],axis=1)
    df_output.drop('Ranking',axis = 1)

    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)

    return df_output

In [15]:
df_preproc = preproc_data(data)
df_preproc.sample(2)

/home/arthur/.local/lib/python3.6/site-packages/ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/arthur/.local/lib/python3.6/site-packages/ipykernel_launcher.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/arthur/.local/lib/python3.6/site-packages/ipykernel_launcher.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/arthur/.local/lib/python3.6/site-packages/ipykernel_launcher.py:123: SettingW

,Ranking,Rating,Number_of_Reviews,Number_of_Reviews_isNAN,Cuisine_Style_NAN,Price_Range_NAN,popular_City,Amsterdam,Athens,Barcelona,...,Style_Xinjiang,Style_Burmese,Style_Fujian,Style_Welsh,Style_Latvian,popular_Cuisine,Price,diff,chain_restor,Ranking_NORM
357,401.0,4.0,774.0,0,0,0,2,0,0,0,...,0,0,0,0,0,3,2,3206,1.0,0.045522
33890,469.0,4.0,409.0,0,0,0,2,0,0,0,...,0,0,0,0,0,3,2,3169,1.0,0.073603


In [16]:
df_preproc = df_preproc.drop('Ranking',axis = 1)

In [19]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df_preproc.drop('Rating', axis = 1)
y = df_preproc['Rating']

In [20]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [21]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [22]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [23]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [24]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21219699999999997


# Вывод:

### Оценки : 0,43 - 0,32 - 0,21 по мере добавления новых параметров.

Удалось улучшить оценку в 2 раза.

Как можно лучше: убрать коррелированные данные, типа количество кухонь и one hot encoding для кухонь; оставить только наиболее значимые параметры

Использумемые параметры
1. one hot для городов и кухонь
2. параметр для NaN 'Number_of_Reviews_isNAN', 'Cuisine_Style_NAN','Price_Range_NAN'
3. параметр популярности города popular_City как one lable encoding
4. количество представленных в ресторане кухонь number_Cuisine
5. параметр наличия популярного блюда/кухни в ресторане popular_Cuisine
6. one label encoding для расброса цены Price
7. параметр, показывающий отклонение между двумя последними оставленными отзывами diff. Вот здесь есть вариант для улучшения модули - пронормировать, чтобы все данные были одного порядка
8. chain_restor -  параметр показывающий входит ли данный ресторан в сеть ресторанов (1 и 0) - one label encoding
9. Ranking_NORM - параметр, который позвоялет выровнять данные между городом и их количеством ресторанов, так как в большом городе много ресторанов , соответственно и будет большая оценка
